In [1]:
# %pip install gensim
# %pip install nltk
# %pip install datasets
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk, datasets
from nltk.tokenize import word_tokenize
from datasets import load_dataset
nltk.download('punkt')

c:\Users\dylan\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>


False

In [2]:
import multiprocessing
multiprocessing.cpu_count()

16

In [ ]:
# loads the yahoo question and answer dataset as a csv
dataset = load_dataset('csv', data_files='train.csv', column_names=['tag', 'q1', 'q2', 'a'])

In [12]:
dataset['train'].info

{'text': '2231378\tThe Lemon Drop Kid , a New York City swindler, is illegally touting horses at a Florida racetrack. After several successful hustles, the Kid comes across a beautiful, but gullible, woman intending to bet a lot of money. The Kid convinces her to switch her bet, employing a prefabricated con. Unfortunately for the Kid, the woman "belongs" to notorious gangster Moose Moran , as does the money. The Kid\'s choice finishes dead last and a furious Moran demands the Kid provide him with $10,000  by Christmas Eve, or the Kid "won\'t make it to New Year\'s." The Kid decides to return to New York to try to come up with the money. He first tries his on-again, off-again girlfriend Brainy Baxter . However, when talk of long-term commitment arises, the Kid quickly makes an escape. He next visits local crime boss "Oxford" Charley , with whom he has had past dealings. This falls through as Charley is in serious tax trouble and does not particularly care for the Kid anyway. As he leav

In [7]:
# select only the answer to train the embedding on for this project
dataset = dataset.select_columns(['a', 'tag'])['train']

In [8]:
testTrain = dataset.shard(num_shards=10, index=0).select_columns(['a', 'tag'])

In [30]:
tagged_data = []
def process(data, idx):
  if data['a']:
    tagged_data.append(TaggedDocument(words=word_tokenize(data['a'].lower()), tags=[str(idx)]))

In [31]:
testTrain.map(process, with_indices=True)

Map: 100%|██████████| 140000/140000 [00:54<00:00, 2575.41 examples/s]


Dataset({
    features: ['a'],
    num_rows: 140000
})

In [33]:
# tagged_data = [data['normalizedBody'] for data in tagged_data.to_list()]
print(len(tagged_data))
type(tagged_data[0])

137512


gensim.models.doc2vec.TaggedDocument

In [83]:
# max_epochs = 100
vec_size = 50
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                min_count=2,
                dm =1,
                workers=14,
                epochs = 100)

model.build_vocab(tagged_data)

# for epoch in range(max_epochs):
    # print('iteration {0}'.format(epoch))
# model.train(tagged_data,
#             total_examples=model.corpus_count,
#             epochs=model.epochs)
#     # decrease the learning rate
#     # model.alpha -= 0.0002
#     # fix the learning rate, no decay
#     # model.min_alpha = model.alpha

# model.save("d2v.model")
# print("Model Saved")

In [84]:
model.train(tagged_data,
          total_examples=model.corpus_count,
          epochs=model.epochs)


model.save("d2v2.model")
print("Model Saved")

Model Saved


In [85]:
ranks = []
second_ranks = []
for doc_id in range(100):
    inferred_vector = model.infer_vector(tagged_data[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(str(doc_id))
    ranks.append(rank)

    second_ranks.append(sims[1])

In [86]:
import collections

counter = collections.Counter(ranks)
print(counter)

Counter({0: 99, 1: 1})


In [9]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v2.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize(testTrain[150]['a'].lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.dv.most_similar([v1])
print(similar_doc)
print(model.dv[similar_doc[1][0]])

V1_infer [-0.6027353   1.2015226   1.1723254  -2.101636   -0.26278505 -1.7776834
  0.310133   -2.3361518   0.12656823 -0.9655682   1.1792357  -0.8823546
 -2.3813164   0.12071148 -0.5840286   0.50054014  1.0074284  -0.5457098
  2.5707026  -1.6397085   2.8314447   0.14812694 -0.5567812   0.79633594
 -1.4670917   0.6348807  -0.23373988  0.7373511  -1.1306167   0.39372474
  1.5446466  -1.1996987   1.2557344  -0.4532859  -0.961303    0.6906949
 -1.4884932  -0.38502076  0.77484995  1.1525539  -1.1173714  -0.488932
 -1.9282116  -0.6406709   2.8561459  -0.9338591  -1.3435836  -1.7458471
 -0.38571405  0.22420828]
[('150', 0.972592294216156), ('23366', 0.6036679744720459), ('10703', 0.5915902256965637), ('70195', 0.5838170647621155), ('108392', 0.5815865993499756), ('30094', 0.5811143517494202), ('87777', 0.5810487866401672), ('92562', 0.5801397562026978), ('20073', 0.576103925704956), ('133844', 0.5759392380714417)]
[ 0.8031868   0.9520606  -0.4840618  -0.3920319   0.15277466 -1.1687264
  0.034

In [10]:
print(testTrain[90]['a'])
print(testTrain[90]['tag'])
print(testTrain[40053]['a'])
print(testTrain[40053]['tag'])

My favorites are Battlefield 2 and Raven shield. Two GREAT games. Espically BF2 with me being a stat fanatic, and all. Plus is has great graphics. Raven shield is an older game, But I like it because I own at it. :} Unreal Tournament 99, An oldie but one of the greatest games ever made. My very first game I got almost 6 years ago.
5
Best: Pink Floyd, worst:any modern band, plus Steve Miller
8


In [11]:
test_data = word_tokenize(testTrain[90]['a'].lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

test_data = word_tokenize(testTrain[40053]['a'].lower())
v2 = model.infer_vector(test_data)
print("V2_infer", v2)

v3 = v1+v2

print("combined: ", v3)

# # to find most similar doc using tags
similar_doc = model.dv.most_similar([v3])
print(similar_doc)
print(model.dv[similar_doc[1][0]])

V1_infer [ 0.14562984 -1.0833215   3.4205241   2.0998147   0.6353861  -3.8460999
 -3.3268867  -0.6087743   0.40012026 -0.23441377  1.8514553   2.764661
  0.505188   -1.7221967   1.337251   -0.46413168  5.8521066   0.4138729
  4.584011   -3.9223018   1.161275    4.1452637   1.6650239  -4.0665455
 -0.7650019  -0.677508    1.3334798  -1.5347203   0.11025764 -0.37048632
 -2.0360367   0.7210217  -0.0644113  -2.2141485   1.7313193   1.8398559
  1.448616    2.3004344   1.0827081   0.4159237   1.0890368  -0.4018157
 -1.881319    1.2954667   2.4869814  -1.7783569   2.6151125  -0.20485084
  0.03439911 -0.39026055]
V2_infer [-3.6923265e-01 -2.7048784e-01  3.3631623e-01  1.1445266e+00
 -1.3930315e+00 -2.0956619e+00 -1.1804276e+00 -1.9083268e-03
  4.7117496e-01 -3.7818754e-01  1.1743270e+00 -4.4428572e-01
 -1.7698908e+00 -1.8597090e-01  3.8770697e-01 -1.0086691e+00
  3.4353476e+00  4.5331183e-01  1.2269120e+00 -2.2764771e+00
  2.5771758e-01  7.5636649e-01 -4.2360881e-01  2.2567016e-01
 -1.2064534e+

In [12]:
for i in range(1,5):
    print(testTrain['a'][int(similar_doc[i][0])])
    print(testTrain['tag'][int(similar_doc[i][0])])

Best: Pink Floyd, worst:any modern band, plus Steve Miller
8
The answer above me is pretty good. And no they are not the best ever. The Packers are; they have the most professional football championships with 12, including 3 Super Bowls.\n\nGuess what Patriot fans, the dynasty is over.
6
it will not be easy, but i think that they will, after all, all of brazil's players are some of the best in the world, like kaka, cafu, dida, adriano, ronaldo, robinho etc.. and  THE CURRENT BEST in the world according to fifa and the public twice in a row ronaldinho, plus their current manajer is the one that has made them achieve their last 2 world cup titles, my bet is that brazils will prevail and come up victourious once again.
6
The Steelers are one of my fav teams, behind the niners and the bucs, but I honestly think the colts were the best team this season. but the Steelers definately had the most heart and played great when it counted
6
